In [1]:
import csv

In [2]:
with open('drugsComTrain_raw.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    i = 0
    data = ""
    data_list = []
    for row in readCSV:
        data_list.append(row[3])
        if i == 400:
            break
        i+=1
# print(data_list)

In [3]:
data_list = data_list[1:]
for i in range(len(data_list)):
    data_list[i] = data_list[i].replace(".","\n")
print(data_list)

['"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"', '"My son is halfway through his fourth week of Intuniv\n We became concerned when he began this last week, when he started taking the highest dose he will be on\n For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him\n) I called his doctor on Monday morning and she said to stick it out a few days\n See how he did at school, and with getting up in the morning\n The last two days have been problem free\n He is MUCH more agreeable than ever\n He is less emotional (a good thing), less cranky\n He is remembering all the things he should\n Overall his behavior is better\n \nWe have tried many different medications and so far this is the most effective\n"', '"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects\n But it contain

In [4]:
data = data.join(i for i in data_list)

In [5]:
data = data.replace("\"\"","\n")
print(data)

"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil
My son is halfway through his fourth week of Intuniv
 We became concerned when he began this last week, when he started taking the highest dose he will be on
 For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him
) I called his doctor on Monday morning and she said to stick it out a few days
 See how he did at school, and with getting up in the morning
 The last two days have been problem free
 He is MUCH more agreeable than ever
 He is less emotional (a good thing), less cranky
 He is remembering all the things he should
 Overall his behavior is better
 
We have tried many different medications and so far this is the most effective

I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects
 But it contained hormone gestodene, whi

In [6]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
    return in_text

def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 10, input_length=max_length-1))
    model.add(LSTM(50))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
#     plot_model(model, to_file='model.png', show_shapes=True)
    return model

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
sequences = list()
for line in data.split('\n'):
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
model = define_model(vocab_size, max_length)
model.fit(X, y, epochs=300, verbose=2)

Using TensorFlow backend.


Vocabulary Size: 3752
Total Sequences: 33375
Max Sequence Length: 161
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 160, 10)           37520     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 3752)              191352    
Total params: 241,072
Trainable params: 241,072
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
 - 105s - loss: 6.5380 - acc: 0.0364
Epoch 2/300
 - 118s - loss: 6.2197 - acc: 0.0412
Epoch 3/300
 - 116s - loss: 6.0349 - acc: 0.0588
Epoch 4/300
 - 113s - loss: 5.7894 - acc: 0.0845
Epoch 5/300
 - 113s - loss: 5.5803 - acc: 0.1042
Epoch 6/300
 - 117s - loss: 5.4224 - acc: 0.1154
Epoch 7/300
 - 119s - loss: 5.2910 -

 - 116s - loss: 2.2395 - acc: 0.4858
Epoch 149/300
 - 116s - loss: 2.2315 - acc: 0.4867
Epoch 150/300
 - 117s - loss: 2.2269 - acc: 0.4872
Epoch 151/300
 - 117s - loss: 2.2244 - acc: 0.4872
Epoch 152/300
 - 117s - loss: 2.2265 - acc: 0.4884
Epoch 153/300
 - 117s - loss: 2.2179 - acc: 0.4890
Epoch 154/300
 - 118s - loss: 2.2316 - acc: 0.4874
Epoch 155/300
 - 117s - loss: 2.2274 - acc: 0.4872
Epoch 156/300
 - 116s - loss: 2.2101 - acc: 0.4906
Epoch 157/300
 - 116s - loss: 2.2029 - acc: 0.4919
Epoch 158/300
 - 116s - loss: 2.2021 - acc: 0.4909
Epoch 159/300
 - 116s - loss: 2.1891 - acc: 0.4950
Epoch 160/300
 - 117s - loss: 2.1985 - acc: 0.4912
Epoch 161/300
 - 117s - loss: 2.1824 - acc: 0.4967
Epoch 162/300
 - 116s - loss: 2.1808 - acc: 0.4975
Epoch 163/300
 - 117s - loss: 2.1756 - acc: 0.4973
Epoch 164/300
 - 116s - loss: 2.1724 - acc: 0.4981
Epoch 165/300
 - 116s - loss: 2.1681 - acc: 0.5001
Epoch 166/300
 - 116s - loss: 2.1660 - acc: 0.5016
Epoch 167/300
 - 117s - loss: 2.1618 - acc: 0

In [10]:
print(generate_seq(model, tokenizer, max_length-1, 'I am', 5))

I am very satisfied with the pilocarpine


In [14]:
print(generate_seq(model, tokenizer, max_length-1, 'I am unwell', 5))

I am unwell very satisfied with the pilocarpine
